In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import time
import json
import os
import io
from os import path

In [2]:
schools = json.load(io.open("..//schools.json", "r"))
conference = "sec"
teamDir = "..//html//rivals//" + conference + "//teams//"
recruitDir = "..//html//rivals//" + conference + "//recruits//"

In [3]:
def checkSchools(recruitSchool, conference):
    
    #clean recruitSchool
    recruitSchoolCleaned = recruitSchool.lower().replace(" ", "").replace("&","")
    #print (recruitSchoolCleaned)
    for school in schools:
        if (conference in school['conference']):
            #print('rivals: ' + school['rivals']))
            #print('recruit: ' + recruitSchoolCleaned)
            if ('rivalsDisplay' in school.keys() and recruitSchool == school['rivalsDisplay']):
                return school['id']

In [4]:
all_recruits = []
error_files = [] 
for file in os.listdir(recruitDir):

    player = {}
    #get file contents and soup it
    recruitSoup = BeautifulSoup(io.open(recruitDir + file, "r", encoding='windows-1252').read(), 'lxml')

    #find the magical html attr
    if (recruitSoup.find("div", class_="profile-block") is not None):
        recruitInfoJson = recruitSoup.find("div", class_="profile-block")['ng-init']
        #this is harsh - but i'm removing an init() and a trailing id which always seems to be x characters long
        recruitInfo = json.loads(recruitInfoJson[5:-57])

        #player info
        #change the rivals school to the proper id
        #for school in schools:
        #    print(school['rivals'] + " and " + recruitInfo['school_name'] )
        #    if (recruitInfo['school_name'] == school['rivals']):
        #        player['school'] = school['id']
        player['school'] = checkSchools(recruitInfo['school_name'],conference)
        player['year'] = recruitInfo['recruit_year']
        player['playerName'] = recruitInfo['full_name']
        player['city'] = recruitInfo['city']
        player['state'] = recruitInfo['state_abbreviation']
        player['highSchool'] = recruitInfo['highschool_name']        
        player['position'] = recruitInfo['position_group_abbreviation']
        player['height'] = recruitInfo['height']
        player['weight'] = recruitInfo['weight']
        player['stars'] = recruitInfo['stars']
        player['nationalRank'] = recruitInfo['national_rank']
        player['positionRank'] = recruitInfo['position_rank']
        player['stateRank'] = recruitInfo['state_rank']

        all_recruits.append(player)
    else:
        error_files.append(file)

In [5]:
with open("..//scraped_json//rivals_" + conference + ".json", "w") as write_file:
    json.dump(all_recruits, write_file)

In [6]:
print(error_files)

[]
